In [1]:
import pandas as pd
from pprint import pprint
import json
from tqdm.auto import tqdm
profiles = json.load(open("profiles.json"))
print(len(profiles))
# De-duplicate profiles
profile_dict = {}
for p in profiles:
    if "profile_id" in p:
        profile_dict[p["profile_id"]] = p
print(len(profile_dict))

7791
7616


In [2]:
pprint(profiles[0]["experience"])

[{'company': {'employeeCountRange': {'end': 500, 'start': 201},
              'industries': ['Health, Wellness and Fitness']},
  'companyLogoUrl': 'https://media-exp1.licdn.com/dms/image/C4D0BAQHe-V0An_SsVA/company-logo_',
  'companyName': 'Diabetes UK',
  'companyUrn': 'urn:li:fs_miniCompany:483655',
  'entityUrn': 'urn:li:fs_position:(ACoAABYLkwwBZlmZPuUAK9ZUaloP8tnTfb1tUeo,1032229291)',
  'geoLocationName': 'London, United Kingdom',
  'geoUrn': 'urn:li:fs_geo:90009496',
  'locationName': 'London, United Kingdom',
  'region': 'urn:li:fs_region:(gb,4573)',
  'timePeriod': {'endDate': {'month': 9, 'year': 2019},
                 'startDate': {'month': 5, 'year': 2017}},
  'title': 'Insight Analyst'},
 {'company': {'employeeCountRange': {'end': 5000, 'start': 1001},
              'industries': ['Management Consulting']},
  'companyLogoUrl': 'https://media-exp1.licdn.com/dms/image/C4D0BAQGkhUlLEL8LTw/company-logo_',
  'companyName': 'PwC New Zealand',
  'companyUrn': 'urn:li:fs_miniCompa

In [3]:
edges = {}
for r in tqdm(profile_dict.values()):
    industries = []
    for i, e in enumerate(r["experience"]):
        if i < (len(r["experience"]) - 1):
            try:
                year = e["timePeriod"]["startDate"]["year"]
                #assert year >= 2010 and year <= 2019
                industry = e["company"]["industries"][0]
                prevIndustry = r["experience"][i + 1]["company"]["industries"][0]
            except:
                continue
            edge_id = prevIndustry + "_" + industry
            if edge_id not in edges:
                edges[edge_id] = {
                    "source": prevIndustry,
                    "target": industry,
                    "val": 0
                }
            edges[edge_id]["val"] += 1

  0%|          | 0/7616 [00:00<?, ?it/s]

In [4]:
df = pd.DataFrame(edges.values()).sort_values(by="val", ascending=False)
df

,source,target,val
7,Management Consulting,Management Consulting,3064
22,Accounting,Accounting,1568
104,Accounting,Management Consulting,679
6,Information Technology and Services,Information Technology and Services,533
49,Banking,Banking,469
...,...,...,...
1477,Food & Beverages,Machinery,1
1474,Building Materials,Real Estate,1
1473,Renewables & Environment,Utilities,1
1472,Hospital & Health Care,Supermarkets,1


In [8]:
%%time
with pd.ExcelWriter('linkedin.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Edges (Industries)', index=False)

CPU times: user 3.23 s, sys: 63.6 ms, total: 3.29 s
Wall time: 3.29 s


In [6]:
# Industries of interest
ioi = ["Management Consulting", "Government Administration"]
df[df.source.isin(ioi) & df.target.isin(ioi)]

,source,target,val
7,Management Consulting,Management Consulting,3064
81,Government Administration,Government Administration,255
60,Government Administration,Management Consulting,177
59,Management Consulting,Government Administration,119
